**GETTING THE DATASET**

In [0]:
!git clone https://github.com/spMohanty/PlantVillage-Dataset

fatal: destination path 'PlantVillage-Dataset' already exists and is not an empty directory.


**IMPORTING OS, NUMPY AND CV2 PACKAGES**

In [0]:
%tensorflow_version 1.x

import numpy as np
import cv2
from os import listdir
from keras.preprocessing.image import img_to_array

TensorFlow 1.x selected.


Using TensorFlow backend.


**RESIZING AND CONVERTING IMAGES TO ARRAY**

In [0]:
default_image_size = tuple((256, 256))

def convert_image_to_array(image_dir):
  image = cv2.imread(image_dir)
  image = cv2.resize(image, default_image_size)   
  return img_to_array(image)

**GETTING THE ROOT DIRECTORY OF DATASET**

In [0]:
directory_root = 'PlantVillage-Dataset/raw/color' 
root_dir = listdir(directory_root)

**LOADING THE IMAGES FROM ROOT / DIFFERENT_DISEASE_PLANT_FOLDER AND CONVERTING IT**

In [0]:
image_list = [] 
label_list = []

for plant_disease_folder in root_dir :
  #getting the inner-image-from-disease-folders
  plant_disease_image_list = listdir(f"{directory_root}/{plant_disease_folder}")
  
  for image in plant_disease_image_list[:5]:
    image_list.append(convert_image_to_array(f"{directory_root}/{plant_disease_folder}/{image}"))
    label_list.append(plant_disease_folder)

**CHECHING THE RESULTS**

In [0]:
print(image_list[:1])
print(label_list[:1])

[array([[[112., 106., 111.],
        [108., 102., 107.],
        [106., 100., 105.],
        ...,
        [127., 151., 157.],
        [118., 144., 150.],
        [ 97., 123., 129.]],

       [[114., 108., 113.],
        [113., 107., 112.],
        [111., 105., 110.],
        ...,
        [122., 146., 152.],
        [103., 129., 135.],
        [ 79., 105., 111.]],

       [[113., 107., 112.],
        [114., 108., 113.],
        [115., 109., 114.],
        ...,
        [116., 140., 146.],
        [ 96., 122., 128.],
        [ 69.,  95., 101.]],

       ...,

       [[138., 130., 131.],
        [138., 130., 131.],
        [138., 130., 131.],
        ...,
        [ 96., 126., 101.],
        [101., 131., 106.],
        [109., 139., 114.]],

       [[137., 129., 130.],
        [137., 129., 130.],
        [137., 129., 130.],
        ...,
        [110., 141., 114.],
        [116., 147., 120.],
        [124., 155., 128.]],

       [[136., 128., 129.],
        [136., 128., 129.],
        [136., 

**LABEL ENCODING**

In [0]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer

label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
n_classes = len(label_binarizer.classes_)

**REGULARIZATION**

In [0]:
np_image_list = np.array(image_list, dtype=np.float16) / 225.0

**TRAIN AND TEST SPLIT**

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.2, random_state = 15)

**CHECKING THE RESULTS**

In [0]:
print(x_train[:1])
print(y_train[:1])

[[[[0.751  0.751  0.778 ]
   [0.6978 0.6978 0.7246]
   [0.8887 0.8887 0.9155]
   ...
   [0.2089 0.1644 0.1644]
   [0.2178 0.1644 0.1644]
   [0.2134 0.16   0.1511]]

  [[0.7866 0.7866 0.8135]
   [0.818  0.818  0.844 ]
   [0.782  0.782  0.809 ]
   ...
   [0.2089 0.1644 0.1644]
   [0.2267 0.1733 0.1733]
   [0.2267 0.1733 0.1644]]

  [[0.6978 0.6978 0.7246]
   [0.991  0.991  1.018 ]
   [0.8535 0.8535 0.88  ]
   ...
   [0.1956 0.1511 0.1511]
   [0.2178 0.1644 0.1644]
   [0.2267 0.1733 0.1644]]

  ...

  [[0.831  0.8354 0.8535]
   [0.844  0.849  0.8667]
   [0.8535 0.858  0.8755]
   ...
   [0.6934 0.6978 0.76  ]
   [0.6    0.6045 0.6665]
   [0.649  0.6533 0.7153]]

  [[0.8354 0.84   0.858 ]
   [0.84   0.844  0.8623]
   [0.849  0.8535 0.871 ]
   ...
   [0.6445 0.649  0.711 ]
   [0.649  0.6533 0.7153]
   [0.662  0.6665 0.729 ]]

  [[0.849  0.8535 0.871 ]
   [0.849  0.8535 0.871 ]
   [0.844  0.849  0.8667]
   ...
   [0.5957 0.6    0.662 ]
   [0.671  0.676  0.738 ]
   [0.6445 0.649  0.711 ]]]]
[[

**MODEL TRAINING TIME**

In [0]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image

In [0]:
EPOCHS = 10
INIT_LR = 1e-3
BS = 32
image_size = 0
width=256
height=256
depth=3

In [0]:
aug = ImageDataGenerator(
    rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, 
    zoom_range=0.2,horizontal_flip=True, 
    fill_mode="nearest")

In [0]:
model = Sequential()
inputShape = (height, width, depth)
chanDim = -1
if K.image_data_format() == "channels_first":
    inputShape = (depth, height, width)
    chanDim = 1
model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(Activation("softmax"))

In [0]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 256, 256, 32)      896       
_________________________________________________________________
activation_7 (Activation)    (None, 256, 256, 32)      0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 256, 256, 32)      128       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 85, 85, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 85, 85, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 85, 85, 64)        18496     
_________________________________________________________________
activation_8 (Activation)    (None, 85, 85, 64)       

In [0]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
history = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // BS,
    epochs=EPOCHS, verbose=1
    )

Epoch 1/10
4/4 [==============================] - 35s 9s/step - loss: 0.0426 - acc: 0.9858 - val_loss: 0.2038 - val_acc: 0.9633
Epoch 2/10
4/4 [==============================] - 36s 9s/step - loss: 0.0351 - acc: 0.9856 - val_loss: 0.2505 - val_acc: 0.9612
Epoch 3/10
4/4 [==============================] - 35s 9s/step - loss: 0.0346 - acc: 0.9882 - val_loss: 0.2331 - val_acc: 0.9619
Epoch 4/10
4/4 [==============================] - 35s 9s/step - loss: 0.0277 - acc: 0.9917 - val_loss: 0.2077 - val_acc: 0.9640
Epoch 5/10
4/4 [==============================] - 35s 9s/step - loss: 0.0322 - acc: 0.9890 - val_loss: 0.2549 - val_acc: 0.9584
Epoch 6/10
4/4 [==============================] - 33s 8s/step - loss: 0.0272 - acc: 0.9897 - val_loss: 0.2088 - val_acc: 0.9612
Epoch 7/10
4/4 [==============================] - 35s 9s/step - loss: 0.0318 - acc: 0.9894 - val_loss: 0.2378 - val_acc: 0.9584
Epoch 8/10
4/4 [==============================] - 37s 9s/step - loss: 0.0273 - acc: 0.9912 - val_loss: 0

In [0]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
#Train and validation accuracy
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend()

plt.figure()
#Train and validation loss
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()

NameError: ignored

In [0]:
print("Calculating model accuracy")
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100}")

Calculating model accuracy
38/38 [==============================] - 3s 85ms/step
Test Accuracy: 96.53739835086623
